<a href="https://colab.research.google.com/github/mr-cri-spy/Movie_seat_enq_bot/blob/main/Movie_bot_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 MB 21.2 MB/s eta 0:00:00


In [9]:
!playwright install

172.5 MiB [] 0% 0.0s172.5 MiB [] 0% 19.9s172.5 MiB [] 0% 7.5s172.5 MiB [] 1% 3.7s172.5 MiB [] 2% 2.6s172.5 MiB [] 3% 2.0s172.5 MiB [] 4% 1.8s172.5 MiB [] 5% 1.9s172.5 MiB [] 6% 1.8s172.5 MiB [] 7% 1.7s172.5 MiB [] 8% 1.6s172.5 MiB [] 9% 1.6s172.5 MiB [] 11% 1.5s172.5 MiB [] 13% 1.4s172.5 MiB [] 14% 1.4s172.5 MiB [] 15% 1.3s172.5 MiB [] 17% 1.2s172.5 MiB [] 19% 1.2s172.5 MiB [] 20% 1.1s172.5 MiB [] 22% 1.1s172.5 MiB [] 23% 1.1s172.5 MiB [] 25% 1.0s172.5 MiB [] 26% 1.0s172.5 MiB [] 27% 1.0s172.5 MiB [] 29% 1.0s172.5 MiB [] 31% 0.9s172.5 MiB [] 32% 0.9s172.5 MiB [] 34% 0.8s172.5 MiB [] 36% 0.8s172.5 MiB [] 38% 0.8s172.5 MiB [] 40% 0.7s172.5 MiB [] 41% 0.7s172.5 MiB [] 43% 0.7s172.5 MiB [] 45% 0.7s172.5 MiB [] 47% 0.6s172.5 MiB [] 49% 0.6s172.5 MiB [] 51% 0.6s172.5 MiB [] 53% 0.5s172.5 MiB [] 55% 0.5s172.5 MiB [] 57% 0.5s172.5 MiB [] 59% 0.5s172.5 MiB [] 60% 0.4s172.5 MiB [] 62% 0.4s172.5 MiB [] 65% 0.4s172.5 MiB [] 66% 0.4s172.5 MiB [] 67% 0.4s172.5 MiB [] 69% 0.3s172.5 MiB [] 70% 0.3s172

In [10]:
# 🧠 1. Setup (only once)
# !pip install playwright
# !playwright install

from playwright.async_api import async_playwright
import json
import asyncio

async def scrape_mysore_movies():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Navigate to Mysore movies
        await page.goto("https://in.bookmyshow.com/explore/movies-mysore")
        await page.wait_for_timeout(3000)

        movie_cards = await page.query_selector_all(".__movie-card")
        movie_data = []

        for card in movie_cards[:5]:  # Limiting for speed
            try:
                name_element = await card.query_selector(".__movie-name")
                name = await name_element.inner_text() if name_element else None
                link_element = await card.query_selector("a")
                link = await link_element.get_attribute("href") if link_element else None
                if name and link:
                    movie_data.append({"name": name, "link": f"https://in.bookmyshow.com{link}"})
            except Exception as e:
                print(f"Error scraping card: {e}")
                continue


        await browser.close()
        return movie_data

# ✅ Test scraping
# Use asyncio.run() to run the async function
data = asyncio.run(scrape_mysore_movies())
with open("live_movie_data.json", "w") as f:
    json.dump(data, f, indent=2)
print("🎬 Movie data saved!")

🎬 Movie data saved!


In [11]:
# Sample structure
movie_live_data = [
    {"theater": "PVR Mysore", "movie": "Kalki 2898 AD", "time": "7:30PM", "seats": 5, "row": "F"},
    {"theater": "INOX Mall", "movie": "Jawan", "time": "8:00PM", "seats": 4, "row": "G"},
    {"theater": "DRC Cinemas", "movie": "Pushpa 2", "time": "6:45PM", "seats": 8, "row": "B"},
]

with open("parsed_showtimes.json", "w") as f:
    json.dump(movie_live_data, f, indent=2)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2")

def generate_answer(query, movie_data):
    movie_text = "\n".join([f"- {d['theater']}: {d['movie']} at {d['time']} – {d['seats']} seats in Row {d['row']}" for d in movie_data])
    prompt = f"""Live Movie Data:\n{movie_text}\n\nUser Query: {query}\n\nAnswer:"""

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example
query = "Where can 5 friends sit together for a movie tonight?"
print(generate_answer(query, movie_live_data))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# app.py
import streamlit as st
import json

with open("parsed_showtimes.json") as f:
    data = json.load(f)

st.title("🎬 Crisbee Live Movie Seat Finder")
query = st.text_input("Ask something like: 'Where can 5 friends sit together tonight?'")

if st.button("Answer"):
    # Dummy reply for now
    for d in data:
        if d["seats"] >= 5:
            st.success(f"{d['theater']} – {d['movie']} at {d['time']} (Row {d['row']})")
